In [ ]:
## THESE ARE THE ONLY LIBRARIES YOU MAY IMPORT!!
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def learnnb(X,Y):
    # X is shape (m,n) (m data points, each with n features).  It has integer values from 0 to maxfeatval (inclusive)
    # Y is shape (m,) (m data points).  It has values of either 0 or 1 (class "0" or class "1")
    maxfeatval = int(X.max().max())
    
    # this function is to return a pair (priorp,condp)
    # where priorp is of shape (2,) and has the prior probability of each of the two classes
    #  and  condp is of shape (n,maxfeatval+1,2) and has the conditional probabilities for the naive Bayes classifier
    
    ## YOUR CODE HERE
    feature_val_cnt = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    class_freq = defaultdict(int)
    row_cnt = len(X)
    classes = np.unique(Y)
    num_of_classes = len(classes)
    for c in classes:
        class_freq[c] = np.count_nonzero(Y == c)
    for feature, c in zip(X, Y):
        feature_Index = 0;
        for feature_val in feature:
            feature_val_cnt[c][feature_Index][feature_val] += 1 
            feature_Index += 1
            
    priorp = np.array([class_freq[int(c)]/row_cnt for c in classes]) 
    number_of_features = len(X[0])
    correction = 1
    condp = np.zeros((number_of_features, maxfeatval+1, num_of_classes))
    for c in classes:
        for j in range(number_of_features):
            for k in range(maxfeatval+1):
                newCnt = feature_val_cnt[int(c)][j][k] + correction;
                condp[j, k, int(c)] = (newCnt)/class_freq[int(c)]
                
    return priorp, condp

In [ ]:
def prednb(X,model):
    # X is of shape (m,n) (m data points, each with n features).
    # model is the pair (priorp,condp), as returned from learnnb
    # should return something of shape (m,) which is an array of 0s and 1s, indicating
    # the predicted (most probable under NB) class for each of the examples in X
    ## YOUR CODE HERE
    (priorp,condp) = model
    m, n = X.shape
    num_of_data_points, num_of_features = X.shape
    result = np.zeros((num_of_data_points,))
    number_of_classes = len(priorp)
    prob = defaultdict(int)
    for i in range(num_of_data_points):
        for cls in range(number_of_classes):
            prob[cls] = priorp[cls]
        for j in range(n):
            for c in range(number_of_classes):
                feature_val = int(X[i,j])
                prob[c] *= condp[j, feature_val, c]
        prob_max = -1;
        prob_max_class = -1;
        for cls, pr in prob.items():
            if pr > prob_max:
                prob_max = pr
                prob_max_class = cls    
        result[i] =prob_max_class
    return result

In [ ]:
def errorrate(predY,trueY):
    if len(predY.shape)>1:
        predY = predY[:,0]
    if len(trueY.shape)>1:
        trueY = trueY[:,0]
    return (predY!=trueY).mean()

model = learnnb(trainX,trainY)
predY = prednb(testX,model)
print(errorrate(predY,testY))